# Using MONAI to unlock clinically valuable insights from Digital Pathology

## Dealing with the size of Whole Slide Images

Today’s image acquisition devices, be they digital pathology slide scanners or lightsheet microscopes, can generate a huge amount of data. This volume of data can make it very challenging to move around, save and load - let alone trying to ingest it into some sort of machine-learning or deep learning algorithm. 

The objective of this workshop is to introduce you to a few tools and techniques that can really help to deliver insights from this rich data without exhausting your system memory or taking eons to run. In fact, you may be surprised to see that outputs from MONAI can be used with the GPU accelerated RAPIDS API to turn some, previously unfeasible, analyses into near-real-time processes.

This workshop will mostly focus on digital pathology, but really, these techniques are very generic and could be applied to data from many different modalities.

Images or volumes can be saved in a variety of formats, some of which are generic and some of which are domain-specific. Additionally, images may be saved and loaded using formats that are based on open-standards or are proprietary to the manufacturers of the device used to capture the image.

The image that we will be using has come from the TCGA archive (https://www.cancer.gov/tcga) and was saved in the .svs format.


## Part 1 - Loading images

Images store a lot of information. Most commonly, images are composed of one or more channels of intensity values across 2 or 3 dimensions. In order to keep file sizes manageable, compression is usually employed. In some cases lossy compression is suitable but for other domains the images need to be lossless. This means that getting all of the pixel data from disk into computer memory can be quite an intense process and without the right tools, techniques and hardware, it can be a slow process. If you have an accelerator such as a GPU you may find that you are unable to utilize its full capabilities because you are unable to feed it data at a sufficiently high rate to keep it busy. 
This first section introduces a few tools that you can use to make best use of the resources available when it comes to loading the data. There are a few factors that come into play here:
- The efficiency of the software algorithm
- The speed of the machine-code that the software is compiled into
- The number of CPU threads or processes used
- The performance of the disk and networking that the data needs to traverse.
- The speed of the CPU
- Any hardware that the CPU supports to accelerate certain processes, such as AVX instructions

For the loading of a variety of biomedical imaging formats, the go-to software has been OpenSlide, which can load formats such as Aperio’s .svs format and many other tiff-based formats. First of all, let’s use OpenSlide to load up one of the images we have to get a feel for the latency involved in loading images at certain resolutions.


In [ ]:
import openslide

# Load the image
slide = openslide.OpenSlide("data/tcga1.svs")

# Get the dimensions at level 0 (Full size)
width, height = slide.level_dimensions[0]

print("Full-Size Image Dimensions - Width = {}, Height = {}".format(width, height))

print("Level Downsamples - {}".format(slide.level_downsamples))

You should see that this image is 87647 x 52434, so that’s 4.6 billion pixels, with 3 color channels - just in one image. To put this into perspective, at a standard display resolution of 120 dots per inch, you’d need a 27 x 20 metre monitor to view this image at full resolution - that's about 2 tennis courts!

For this very reason, these types of image are often saved in formats that allow the image to be loaded at a lower resolution or provide a means of only loading a small sub-region of the image.

You should also notice that the Whole Slide Image doesn't just contain the full resolution image but also a pyramid of resolutions (The Level Downsamples). In this case, along with the full resolution image, there are also 4x, 16x and 32x down-sampled versions of the image. This permits viewers to load the image at lower resolutions for a broad overview and then the user can choose to zoom in to specific regions, using the higher resolution versions. The more levels that the pyramid contains, the smoother the zooming will be - at the cost of larger file sizes.

As we will see later in this notebook, these lower resolution views can also be used for eliminating the empty regions from processing by applying some sort of thresholding function to them and only selecting tiles for inference (or training) from the foreground (tissue) regions.


![Pyramid](images/pyramid.png)

Next we are going to load the image at the lowest resolution in the pyramid - by specifying _slide.level_count-1_ for the _level_ parameter

In [ ]:
import matplotlib.pyplot as plt

# get height and width at the lowest level resolution
w_thumbnail, h_thumbnail = slide.level_dimensions[slide.level_count-1]

# Load up the image data at the lowest resolution - to preview it
img = slide.read_region((0,0), slide.level_count-1,(w_thumbnail, h_thumbnail))
print("Reduced-Size Image Dimensions - Width = {}, Height = {}".format(w_thumbnail,h_thumbnail))

# Use Matplotlib to display the thumbnail view of the image
plt.figure(figsize=(10,10))
plt.imshow(img)
plt.title('tcga1.svs')
plt.show()

The image displayed shows that many of the image pixels are actually not very informative. A lot of the image is white background, since this image contains a single tissue slice that is centred on the slide. 

So let’s investigate the loading time for different resolutions of this image. In the cells below you will see some skeleton code which you need to flesh out to measure the time it takes to load the image at each of the resolutions that it contains. You should complete the code so that it plots the times for each resolution.

time_loading_at_resolution is a function that takes a slide and a reduction level and returns the time it takes to load the image data at that resolution

Check the [solution](solutions/loading_at_resolution.py) if you get stuck


In [ ]:
from timeit import default_timer as timer

def time_loading_at_resolution(slide, level): # slide 
    
    start = timer()
    
    # TODO - insert code to print out the dimensions and load the image at the specified level

    end = timer()
    
    return end - start

When you have completed the code in the cell above and run it, you can test it by running the cell below. It should print out a range of load times for different resolutions

In [ ]:
# Initialise a list to hold the loading times
times = [0] * (slide.level_count-1)

# Now call the timing function for a range of possible resolutions
for i in range(slide.level_count-1,0,-1):
    times[i-1] = time_loading_at_resolution(slide,i)
    print("Time at resolution reduction level {} = {}".format(i, times[i-1]))
    
print("Completed")

If that works as expected, you should now have the load times in the array we created, which we can plot out by running the cell below. Note that we are not loading the image at full resolution (level 0) because that would take a long time (and requires sufficient memory).

In [ ]:
# Now plot the load times
downsample_factor = slide.level_downsamples[1:4]

plt.plot(downsample_factor, times, '-ok')
plt.xlabel("Reduction Factor")
plt.ylabel("Load Time (s)");
plt.yscale("log")

plt.show

So, because the number of pixels doubles for each doubling of a dimension size, for a 2D image, the load time is quadratically related to the downsample level. As mentioned, we avoided loading the image at reduction level 0 because, according to this trend, it might take several minutes at full resolution.

So, what can we do to reduce this load time? One technique that is often used to speed up many different types of operation is to use multi-threading. Multi-threading is a technique in which the process or program running your code spawns multiple sub-processes, known as threads, which can then operate in parallel, reducing the overall time to perform certain operations. In this case, we could get multiple threads loading different parts of the image. However, Python has a mechanism to prevent issues caused by concurrent execution (e.g. data races), known as the Global Interpreter (aka GIL) and this can prevent optimal performance.

## Introducing CuCIM

When dealing with much larger images, it is necessary to utilise as much of the available compute power that we have to run in parallel, otherwise it can be difficult to keep the GPU busy all the time. The problem we face here is that it is not just the Python GIL that we are working with but OpenSlide itself is not especially fast at this sort of operation. For this reason, the cuCIM library was  added to the RAPIDS platform and is also used in MONAI. cuCIM offers similar capabilities to Openslide but has been optimised for the scenario we are exploring. The API is similar, but not exactly the same as OpenSlide, so you can see that, to do what we did before, we will need to amend the loading code slightly. Have a look at the code cell below to see how to get the image dimensions at a specific resolution and load the image.

N.B. When loading a specific region of interest at a reduction level > 0, you need to supply the x and y coordinates at the full resolution, whereas the width and height should be supplied at the reduced size. See the [documentation](https://docs.rapids.ai/api/cucim/stable/api.html#module-cucim.CuImage) for more details

In [ ]:
from cucim import CuImage

input_file = "data/tcga1.svs"
# load the image header
wsi = CuImage(input_file)

# Get the resolution meta data
sizes=wsi.metadata["cucim"]["resolutions"]
levels = sizes["level_count"]

# Get the dimensions at the lowest resolution level
wt = sizes["level_dimensions"][levels-1][0]
ht = sizes["level_dimensions"][levels-1][1]

# Load the image data at this resolution
wsi_thumb = wsi.read_region(location=(0,0), size=(wt,ht), level=levels-1)

plt.figure(figsize=(10,10))
plt.imshow(wsi_thumb)
plt.title('tcga1.svs')
print(wt,ht)
plt.show()

Now we can compare the performance of image loading using OpenSlide and cuCIM. In the code cell below add the necessary steps for cucim to load the image at the specified resolution ([solution](solutions/load_resolution.py))

In [ ]:
from timeit import default_timer as timer

def time_loading_at_resolution(level, use_cucim):
    
    start = timer()

    if use_cucim:
        sizes=wsi.metadata["cucim"]["resolutions"]

        # Get the dimensions at the lowest resolution level
        wt = sizes["level_dimensions"][level][0]
        ht = sizes["level_dimensions"][level][1]

        # TODO insert code to load the image at the specified resolution reduction level  
        # and with the full width and height at that resolution
        wsi_thumb = wsi.read_region(location=(0,0), size=(wt,ht), level=level)
    else:
        width, height = slide.level_dimensions[level]
        img = slide.read_region((0,0), level, (width, height))

    end = timer()
    
    return end - start

Once you have completed and run the code cell above, you can run the code below to test the function and generate some load times to compare

In [ ]:
# Now call the timing function for each of the possible resolutions
cu_times = [0] * (slide.level_count-1)
times = [0] * (slide.level_count-1)

print("Using cuCim...")
for i in range(slide.level_count-1,0,-1):
    cu_times[i-1] = time_loading_at_resolution(i,True)
    print("Time at resolution reduction level {} = {}".format(i, cu_times[i-1]))

print("Using OpenSlide...")
for i in range(slide.level_count-1,0,-1):
    times[i-1] = time_loading_at_resolution(i,False)
    print("Time at resolution reduction level {} = {}".format(i, times[i-1]))

print("Completed!")

When it says 'Completed', let's plot that out 

In [ ]:
reduction_factor = slide.level_downsamples[1:4]

plt.plot(reduction_factor,times, '-ok')
plt.plot(reduction_factor,cu_times, '-or')
plt.xlabel("Reduction Factor")
plt.ylabel("Load Time (s)")
plt.yscale("log")

plt.show

So, you should notice that CuCIM is about an order of magnitude faster at loading the image data (note the log scale on the y-axis).

cuCIM includes a feature (since v21.12.1) that actually uses multiple threads internally to load an image. This is a much more efficient and cleaner way of quickly loading an image. It requires no Python GIL workarounds and uses far fewer resources. Let's compare it with our Python implementation.

Please note that we are loading a very large image and so there is a chance that we will run out of RAM when loading images at full resolution in this memory-limited environment. If this happens you will most likely see an error message pop up telling you that the kernel just re-launched. If this happens, it will actually remove all the current data from RAM from the previous cells and it will probably work if you try again (No need to re-run any previous cells).

In [ ]:
%%time
from cucim import CuImage

input_file = "data/tcga1.svs"
sizes=wsi.metadata["cucim"]["resolutions"]
width = sizes["level_dimensions"][0][0]
height = sizes["level_dimensions"][0][1]
img = wsi.read_region((0,0),(width, height), 0, num_workers=16)
print(img.shape)
del(img) # reclaim the memory!

Now we can try loading the image with OpenSlide but use different threads to load different parts of the image. Although the run time is slower than cuCIM, this is not a particularly fair comparison since in the simplistic OpenSlide code, there was no ability to actually assign the sub regions loaded into a single global array. Also, the speed-up you get is affected by the number of CPU cores available and the Cloud instances that are hosting these sessions typically only have a handful of virtual cores. You will see better performance gains on a higher-spec workstation or server.

In [ ]:
%%time
import threading
import time

# You can also try changing the number of threads and see what effect
# it has on the overall run time
num_threads = 16
level = 0

class loaderThread (threading.Thread):
    def __init__(self, threadID):
        # Class initialisation - set its ID
        threading.Thread.__init__(self)
        self.threadID = threadID

    def run(self):
        print("Starting thread {}".format(self.threadID))
        start = timer()
        width, height = slide.level_dimensions[level]
        x = (width // num_threads) * (1-self.threadID)
        img = slide.read_region((x,0), level,(width//num_threads, height))
        end = timer()
        print("Exiting thread {}, running time = {}".format(self.threadID, str(end-start)))

threads = []

for i in range(num_threads):
    # Create new threads
    thread = loaderThread(i)
    thread.start()
    threads.append(thread)

# Wait for all threads to complete
for t in threads:
    t.join()
    
print("Exiting Main Thread")

So, despite the unfair comparison, cuCIM should have loaded the whole image in a shorter time because it is using multiple threads to load the data concurrently and because this is happening at C++ layer, there is no GIL problem to slow things down. You can also see that the output reports the total CPU time as ~50 seconds - which is how long it might have taken using a single thread.

What is also useful is that although cuCIM uses concurrent threads to load separate regions, it stitches them all together into one array

N.B. If you do keep running out of memory, you can load the image at a reduction level of 1 instead of 0 (full resolution).

This section should have given you a good grasp of how much of a difference the combination of a decent image loader and some threading or multi-processing can make.

**Using Dask instead of doing threading ourselves**

In this next step we are going to introduce [DASK](https://docs.dask.org/en/stable/https://docs.dask.org/en/stable/), which is a very useful tool for breaking large tasks into lots of smaller chunks to reduce overall latency. For many Python developers, this is a preferable alternative to directly working with multiprocessing and multithreading APIs in the previous exercises. DASK provides features that resemble some of these functions, but it also provides a swathe of other benefits including:

* A rich set of visualization tools to monitor the status of your running code
* Integrations and compatibility with many other tools from the Data Science ecosystem
* Abstractions that provide powerful but simple to use concurrency

When it comes to concurrency, DASK provides two main tools - Futures and Delayed functions.

Futures are used to asynchronously process results, with the results becoming available when the computation has completed. Delayed functions are used to 'lazily' compute values, as the results of prior computations or inputs become available.

Let's see how we could use this to threshold a Whole Slide Image (i.e. remove empty background regions)

First, we create a Dask cluster...

In [ ]:
from dask.distributed import as_completed
from dask.distributed import Client, LocalCluster

# Setup a local cluster.
cluster = LocalCluster(dashboard_address= 8789, processes=True)
client = Client(cluster)
client


Next we define a function to check whether there is any tissue in a passed-in image tile. Variance is a fairly good proxy and is also invariant to intensity, which means that we don't need to establish a unique threshold for each individual image.

In [ ]:
# evaluates whether the block contains tissue to analyse
def threshold(arr, threshold_value=80):
    
    # check whether there is sufficient variance in the input
    if arr.flatten().var() > threshold_value:
        return True
    else:
        return False

Next we define a patch size and get the downsample factor used at level 1. There is no need to threshold at full resolution in this case.

We also define a function that takes a list of coordinates and evaluates each tile using the previously defined threshold function. If the result is above threshold, then the coordinates get added to a results list, otherwise they are discarded.

The compile_results function waits for all the Dask processes to return results and adds them to a global list. 


In [ ]:
import numpy as np

patch_size = 164
input_file = "data/tcga1.svs"
reduction = int(wsi.metadata["cucim"]["resolutions"]["level_downsamples"][1])

# iterate over a set of regions from which to threshold
def process_patch(start_loc_list):
    
    # load the image header
    wsi = CuImage(input_file)
    res = []
    
    for start_loc in start_loc_list:
        region = np.array(wsi.read_region(location=start_loc, size=[patch_size//reduction , patch_size//reduction], level=1))
        if threshold(region):
            res.append((start_loc[0], start_loc[1]))
        
    return res

# As the results are processed, put them into a list
def compile_results(futures):
    patches = []

    for future in as_completed(futures):
        res1 = future.result()
        if res1:
            for patch in res1:
                patches.append(patch)
                
    return patches

Now we can actually execute the thresholding steps. We build a list of coordinates that need to be evaluated and then split this list into _num_chunks_ smaller lists and let Dask distribute them between the worker processes in the cluster that we created. The results are accumulated asynchronously by the _compile_results_ function

Note that this will take a few tens of seconds to execute. If you load the Dask Toolbox from the left toolbar, you can see the progress of the computation. You'll need to add the URL of the Dask Cluster into the Search box which will be something like http://...courses.nvidia.com:8789 (use the URL of the cloud instance from your browser and append the :8789 port) 

In [ ]:
%%time
num_chunks = 32

w = sizes["level_dimensions"][0][0]
h = sizes["level_dimensions"][0][1]

start_loc_data = [(sx, sy)
                  for sy in range(0, h, patch_size)
                      for sx in range(0, w, patch_size)]

chunk_size = len(start_loc_data) // num_chunks

start_loc_list = [start_loc_data[i:i+chunk_size]  for i in range(0, len(start_loc_data), chunk_size)]
future_result1 = list(client.map(process_patch, start_loc_list))
patches = compile_results(future_result1)
                 
print("Number of patches found = {}".format(len(patches)))

Let's plot that out now by creating a binary mask and setting the pixels to 1 for all patches that were returned.

In [ ]:
w = sizes["level_dimensions"][1][0]
h = sizes["level_dimensions"][1][1]

mask = np.zeros((h//patch_size,w//patch_size),dtype=int)

print(mask.shape)

for patch in patches:
    j = patch[0]//(patch_size*reduction)
    i = patch[1]//(patch_size*reduction)
    if i< h//patch_size and j < w//patch_size:
        mask[i,j]=1
    
plt.figure(figsize=(10,10))
plt.imshow(mask)
plt.title('threshold mask')
plt.show()

Hopefully you can see the outline of the tissue from the WSI. As you will see in the next notebook, this sort of capability could be used during inference or training to eliminate regions of the WSI from unnecessary inference.

**Bonus Material - Toy Dask Example**

Let's look at a toy example. Imagine that we want to sum a series of integers. Naively, you'd have to iterate over each element one at a time adding each element to the running total. The run time would be a factor of the number elements. A better way would be to concurrently add every other element to its neighbour iteratively until there is only one element left. This would bring the runtime down to log(N) time. By providing a few basic commands you can let Dask figure out the execution graph for you. Let's look at a concrete example

We can write the code to do the adding for us using a Dask Delayed function. This means that before the result is calculated a graph is constructed and Dask will map this graph onto the available compute (e.g. Processes, Threads or GPUs)

In [ ]:
import dask
from dask import delayed

@dask.delayed
def add(x, y):
    return x + y

a = [i+1 for i in range(16)]
b = []

while len(a)>1:
    for i in range(0,len(a),2):
         b.append(add(a[i],a[i+1]))
    a=b
    b=[]
    
result = a[0]
   
result.visualize()

At this point, no computation has been done - just the graph construction. By doing this up-front, a more efficient graph can be created. You can see that the graph shows how the additions at each phase can be done in parallel , but also how each subsequent addition depends only on its ancestors. To actually do the computation, we need to execute a compute() command

In [ ]:
%%timeit
result.compute()

Note that, in this toy example, the overhead of organizing the concurrency would far outweight any gains. This technique is really only suitable for larger problems. So, make sure you don't prematurely optimize anything ("the root of all evil" according to Donald Knuth!)